# Matrix Encryption
Makenna Worley and Nic Van Der Werf

An exploration into encrypting data through matrices transformations and modular arithmetic

In [30]:
# Importing and declaring global variables
import numpy as np
import math
import random

from ipywidgets import widgets
from IPython.display import display

MODULO = 59
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?'\n"
# print(len(ALPHABET))

KEY_MATRIX_2x2 = np.array([
    [3, 2],
    [5, 7]
])

text1 = open('text/one_line.txt').read()
text2 = open('text/two_lines.txt').read()
text3 = open('text/our_super_secret_message.txt').read()

Functions that will help us with encrypting and decrypting our secret messages!

Generates a random 2 x 2 Key Matrix. Checks to make sure Greatest Common Denominator is 1 and that it is invertible with the modulo.

In [2]:
def generate_key_matrix_2x2(modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(2, 2))

        # Calculating the determinant with Mod
        a, b = key_matrix[0]
        c, d = key_matrix[1]
        det = (a*d - b*c) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            print("Failed Iteration")

Inverts a 2 x 2 Matrix with the modulo using integers 

In [3]:
def inverse_2x2_mod(matrix, modulo):
    a, b = matrix[0]
    c, d = matrix[1]
    det = int((a*d - b*c) % modulo)
    if math.gcd(det, modulo) != 1:
        raise ValueError("Matrix not invertible modulo {}".format(modulo))
    det_inv = pow(det, -1, modulo)
    return (det_inv * np.array([[d, -b],
                                [-c, a]])) % modulo

Encrypts plain text using a 2 x 2 key matrix. by converting text to integers based on the provided alphabet, creating a matrix of those integers, getting the dot product with the key, then converting back to text. Modulo is used to ensure all digits are in the range of the alphabet.

In [4]:
def encrypt(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range

    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]

    # Pad if necessary
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    plaintext_matrix = np.array(numbers).reshape(-1, 2).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T

    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix

Decrypts plain text using a 2 x 2 inverse key matrix by converting text to integers based on the provided alphabet, creating a matrix of those integers, getting the dot product with the inverse key, then converting back to text. Modulo is used to ensure all digits are in the range of the alphabet.

In [5]:
def decrypt(ciphertext, inverse_key_matrix, modulo, alphabet):
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    cipher_matrix = np.array(numbers).reshape(-1, 2)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('*')
    return ''.join(plaintext_chars).rstrip('A')

Generates a random n x n Key Matrix. Checks to make sure Greatest Common Denominator is 1 and that it is invertible with the modulo.


In [6]:
def generate_key_matrix_nxn(n, modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(n, n))
        det = int(round(np.linalg.det(key_matrix))) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_nxn_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            # All matrices must be invertible
            pass

Inverts a n x n matrix with the modulo using Gaussian Elimination

Inverting matrices with >2 rows and columns requires GCD since it is not guaranteed that rows and columns will be the same number, so we need to look at the coprime of them

In [7]:
# Inverting matrices with >2 rows and columns requires GCD since it is not guaranteed that rows and columns will be the same number, so we need to look at the coprime of them
def inverse_nxn_mod(matrix, modulo):
    A = np.array(matrix, dtype=int) % modulo
    n = A.shape[0]

    I = np.eye(n, dtype=int)

    # Perform Gaussian elimination
    for i in range(n):
        # Find pivot (row with non-zero element in column i)
        for r in range(i, n):
            if A[r, i] % modulo != 0:
                # Swap pivot row if necessary
                if r != i:
                    A[[i, r]] = A[[r, i]]
                    I[[i, r]] = I[[r, i]]
                break
        else:
            # No pivot found, matrix not invertible
            raise ValueError("Matrix not invertible modulo {}".format(modulo))

        # Normalize pivot row
        inv_pivot = pow(int(A[i, i]), -1, modulo)
        A[i] = (A[i] * inv_pivot) % modulo
        I[i] = (I[i] * inv_pivot) % modulo

        # Eliminate other rows
        for r in range(n):
            if r != i:
                factor = A[r, i] % modulo
                if factor != 0:
                    A[r] = (A[r] - factor * A[i]) % modulo
                    I[r] = (I[r] - factor * I[i]) % modulo

    return I % modulo

Encrypts plain text using a n x n key matrix. by converting text to integers based on the provided alphabet, creating a matrix of those integers, getting the dot product with the key, then converting back to text. Modulo is used to ensure all digits are in the range of the alphabet.

Matrix created from text is dynamically sized so that the dot product can be taken with the n x n key


In [8]:
def encrypt_nxn(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    n = key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range

    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]

    # Pad if necessary
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    plaintext_matrix = np.array(numbers).reshape(-1, n).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T

    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix

Decrypts plain text using a 2 x 2 inverse key matrix by converting text to integers based on the provided alphabet, creating a matrix of those integers, getting the dot product with the inverse key, then converting back to text. Modulo is used to ensure all digits are in the range of the alphabet.

Matrix created from text is dynamically sized so that the dot product can be taken with the n x n inverse key


In [9]:
def decrypt_nxn(ciphertext, inverse_key_matrix, modulo, alphabet):
    n = inverse_key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    cipher_matrix = np.array(numbers).reshape(-1, n)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('?')
    dec = ''.join(plaintext_chars)
    return dec.rstrip('A')

Functions for creating and testing random strings with a 2 x 2 key matrix

In [10]:
def generate_random_string(alphabet, l):
    return ''.join(random.choice(alphabet) for _ in range(l))


def test_encryption_decryption(l):
    plaintext = generate_random_string(ALPHABET, l)
    ciphertext, ciphertext_matrix = encrypt(plaintext, KEY_MATRIX_2x2, MODULO, ALPHABET)
    decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
    return plaintext, decrypted_text, plaintext == decrypted_text

First testing our matrix to make sure it is invertible and therefore useable for our purposes.

In [16]:
inverse_key_matrix = inverse_2x2_mod(KEY_MATRIX_2x2, MODULO)

for idx in range(1, 51):
    plaintext, decrypted_text, result = test_encryption_decryption(200)
    if result:
        print(f"Test {idx}: True")
    else:
        print(f"Test {idx}: False")
        print(f"  Original: {plaintext}")
        print(f"  Decrypted: {decrypted_text}")

Test 1: True
Test 2: True
Test 3: True
Test 4: True
Test 5: True
Test 6: True
Test 7: True
Test 8: True
Test 9: True
Test 10: True
Test 11: True
Test 12: True
Test 13: True
Test 14: True
Test 15: True
Test 16: True
Test 17: True
Test 18: True
Test 19: True
Test 20: True
Test 21: True
Test 22: True
Test 23: True
Test 24: True
Test 25: True
Test 26: True
Test 27: True
Test 28: True
Test 29: True
Test 30: True
Test 31: True
Test 32: True
Test 33: True
Test 34: True
Test 35: True
Test 36: True
Test 37: True
Test 38: True
Test 39: True
Test 40: True
Test 41: True
Test 42: True
Test 43: True
Test 44: True
Test 45: True
Test 46: True
Test 47: True
Test 48: True
Test 49: True
Test 50: True


Now let's look at our encrypting a message using our predefined KEY_MATRIX_2x2

In [17]:
MESSAGE = "Hello my name is Makenna"

inverse_key_matrix = inverse_2x2_mod(KEY_MATRIX_2x2, MODULO)
print(f"Key Matrix:\n{KEY_MATRIX_2x2}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(MESSAGE, KEY_MATRIX_2x2, MODULO, ALPHABET)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")
print(f"Condition: {np.linalg.cond(KEY_MATRIX_2x2)}")

original_text = MESSAGE

print("Cipher Text Matrx: \n", ciphertext_matrix)
print("\n\n")

print("Original Text :   ",original_text)
print()
print("Cipher Text   :   ",ciphertext)
print()
print("Decrypted Text:   ",decrypted_text)


Key Matrix:
[[3 2]
 [5 7]]
Inverse Key Matrix:
[[ 6 32]
 [21 11]]
Ciphertext Matrix:
[[22  9]
 [ 8 31]
 [47 33]
 [37  9]
 [57  2]
 [36 42]
 [17 42]
 [13  6]
 [ 3 49]
 [32 28]
 [50 10]
 [51 23]]
Ciphertext: WJIfvhlJ'CkqRqNGDxgcyKzX
Condition: 7.780565547548909
Cipher Text Matrx: 
 [[22  9]
 [ 8 31]
 [47 33]
 [37  9]
 [57  2]
 [36 42]
 [17 42]
 [13  6]
 [ 3 49]
 [32 28]
 [50 10]
 [51 23]]



Original Text :    Hello my name is Makenna

Cipher Text   :    WJIfvhlJ'CkqRqNGDxgcyKzX

Decrypted Text:    Hello my name is Makenna


Example with the generating of a 2 x 2 random key. 

In [29]:
MESSAGE = "Hello my name is Nic"

key_matrix, inverse_key_matrix = generate_key_matrix_2x2(MODULO)

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(MESSAGE, key_matrix, MODULO, ALPHABET)

decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)

original_text = MESSAGE

print("Cipher Text Matrx: \n", ciphertext_matrix)
print("\n\n")

print("Original Text :   ",original_text)
print()
print("Cipher Text   :   ",ciphertext)
print()
print("Decrypted Text:   ",decrypted_text)

Key Matrix:
[[38 22]
 [11  9]]
Inverse Key Matrix:
[[29 34]
 [17 11]]
Cipher Text Matrx: 
 [[41 52]
 [37 32]
 [ 9 23]
 [ 7 42]
 [ 2 38]
 [54 38]
 [42 31]
 [18  3]
 [20 40]
 [20 36]]



Original Text :    Hello my name is Nic

Cipher Text   :    p lgJXHqCm,mqfSDUoUk

Decrypted Text:    Hello my name is Nic


In [33]:
MESSAGE = text1
KEY_MATRIX_SIZE = 7

key_matrix, inverse_key_matrix = generate_key_matrix_nxn(KEY_MATRIX_SIZE, MODULO)

ciphertext, ciphertext_matrix = encrypt_nxn(MESSAGE, key_matrix, MODULO, ALPHABET)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decrypt_nxn(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)

Plain Text Length: 93
Ciphertext Length: 98
Key Matrix:
[[24  3 40 18 15 46  4]
 [40 27 14 43 46  8  2]
 [18 35 19 58 51 25 32]
 [13 42  0 38 26 48  5]
 [29 14 12 51 27 48 16]
 [32 42 14 45 36 43 33]
 [ 0 34 21 34 24 12 31]]
Inverse Key Matrix:
[[ 3 57 35 43 55 11 46]
 [41 16 16 43 31  9 36]
 [30 20 43 31 36 14 49]
 [23 42 19 31  6 14  5]
 [35 39 20 28 42 45  1]
 [46 16 36 53 48 28 45]
 [53 33 42  6 40 30 35]]
Ciphertext: pFppuAJbDanKUHKhBZs!aPEsOft?Ksyjkb!.LcgDuE'YpJjmtEOPwOu'B!FVk!eXYhhSDMr'LibYDuQizUYV oinJvfacP!LjV


Button(description='Show Original Text', style=ButtonStyle())

Output()

Button(description='Show Decrypted Text', style=ButtonStyle())

Output()

In [34]:
MESSAGE = text2
KEY_MATRIX_SIZE = 23

key_matrix, inverse_key_matrix = generate_key_matrix_nxn(KEY_MATRIX_SIZE, MODULO)

ciphertext, ciphertext_matrix = encrypt_nxn(MESSAGE, key_matrix, MODULO, ALPHABET)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decrypt_nxn(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)

Plain Text Length: 93
Ciphertext Length: 69
Key Matrix:
[[38 30 24  4 51 10 24 33 27 14 30 26 58 33 39 24 22  9 53  5 47 20 22]
 [24 36 16 35 37  2 58 13 43 47 21 25  2 37  5 29 28 33 24 42 24  3 55]
 [10 29  5 17  2 37 39 28  9 48 55 19 17 29  9 53 23 39 23  6 12 47 12]
 [13  3  3 47 44 40 21 28 11 23 20 58 18 28 38 28 36 42 42 34 54 51 25]
 [ 9 41 48 19 24 53  4 40 29 45 26 20 52 22 36 17 51  2 36 11 58 11 57]
 [49 31 42 17 12 58 27 54 12 33 57 47 22  0 43 40 27 31 16 34 43  7 14]
 [ 4  5 12 46 13 56 14 26 52  4 18 29 46 53 39 55 37 11 30 14 16 33 55]
 [13 23 44 30 20  6 36 11 45 37 17 55 46 48 39 35 49 14 32  4 27 19 26]
 [26 57  9  3 42  8 47 37 47 54 40 29 52 43 17 31 49 10 22 17 41 34 16]
 [44  4 27 14 39 53 27 34 54 29 46 28 50 40  2 23 46  3 34 41 54 12 16]
 [ 1 26 12 16 12 35 25 21 20 49 16 36 49 43 37 49 15 45 14 47 38 52 15]
 [34  2 58  3 14 30 53 55 16 17  5 13 32 55 48  3 22 50 36 34 54 46 55]
 [35 32 31  5 15 40  0 34 19 42 31 57 54 35  9 29 17  4  5 51 25 14 57]
 [24 23 

Button(description='Show Original Text', style=ButtonStyle())

Output()

Button(description='Show Decrypted Text', style=ButtonStyle())

Output()

In [35]:
MESSAGE = text3
KEY_MATRIX_SIZE = 100

key_matrix, inverse_key_matrix = generate_key_matrix_nxn(KEY_MATRIX_SIZE, MODULO)

ciphertext, ciphertext_matrix = encrypt_nxn(MESSAGE, key_matrix, MODULO, ALPHABET)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decrypt_nxn(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)

Plain Text Length: 93
Ciphertext Length: 1800
Key Matrix:
[[31  2 57 ... 53 23 35]
 [43 31 22 ... 42 49 46]
 [12 31  7 ...  7 40 39]
 ...
 [ 5  1 55 ... 24 50 53]
 [43 27 31 ...  8 31 25]
 [42  4 21 ... 24 45 25]]
Inverse Key Matrix:
[[39  8 12 ... 34 30 23]
 [45 55 54 ...  0  7 35]
 [14 42 41 ... 40 44 31]
 ...
 [29 10 13 ... 51 18 49]
 [15 45 48 ...  0 55 14]
 [28 16 39 ... 54 40 44]]
Ciphertext: GCQifuiAS
UtyJyXW?WptpJ'jMgGCz.boBpxyoS'?oqulTVSCcD'nkbnkMkHQIfUlVst!lPMIVNbxd?Ox,GcEw?jtfStVWRdB?z!YUKZQnlHsc!.Crla!iWAPrhygZkt?pBbFlTJGlz.WBFI'xEtkG!PfxcmC
!fyAQKCQvxahCTTACSbDl.y,OfYCaHErZxgcwowhrWk'LuQO ,vtpguDP.q'Xgrpuhz!LXBB
,lZuai.qGojLqXxItur,qoqyx'stGswhRJrYNuL'RFlGnzD?qEkbvVAwGzamBP?e
DwWngzg?guhIBJAPyRv ESg!
rZhkDRoxWxjKdxkGYiuEtsuvQNiOpDRtbxNzW
hDpJiDaDJMxVYtmjZonkUcu
CiOiN.BDUSsddZEP?Msvp,WxCbUWdhgL?KukTZcod.KbkuHRPaBk,QA,cjRjt hdibOVtRfHkOCEbdKmxZ
NLbSl.Hgh
YjrgaVtIhwnSNktZVNcM'kL,P!hCj
!ppHG!yYTzSV!lThNtf.plsfxCrKk KjeAM. uOCxY

JYwWBCcWj'pSsmlYa
JhOcSdgld KQCnYfTPUF'pAAfnGPo!

Button(description='Show Original Text', style=ButtonStyle())

Output()

Button(description='Show Decrypted Text', style=ButtonStyle())

Output()